In [0]:
-- SELECT balance_source, count(*) AS rows
-- FROM tp_finance.silver.contract_facility_monthly_canonical
-- GROUP BY balance_source;

-- SELECT *
-- FROM tp_finance.silver.contract_facility_monthly_canonical
-- WHERE is_mismatch = true
-- ORDER BY balance_diff DESC;



-- SELECT facility_id, min(month) AS first_hist_month, max(month) AS last_hist_month, count(*) AS months
-- FROM tp_finance.silver.contract_facility_monthly_canonical
-- WHERE balance_source = 'HIST_SNAPSHOT'
-- GROUP BY facility_id;

-- SELECT
--   count(*) AS overlap_months,
--   sum(CASE WHEN is_mismatch THEN 1 ELSE 0 END) AS mismatched_months
-- FROM tp_finance.silver.contract_facility_monthly_canonical
-- WHERE lti_closing_balance IS NOT NULL AND hist_closing_balance IS NOT NULL;



-- SELECT count(*) FROM tp_finance.gold.fact_facility_monthly;
-- SELECT count(*) FROM tp_finance.gold.fact_contract_monthly_summary;
-- SELECT count(*) FROM tp_finance.gold.dim_month;


SELECT
  s.customer_id, s.contract_id, s.month,
  s.total_drawn,
  f.total_drawn_check,
  s.total_repaid,
  f.total_repaid_check,
  s.closing_balance,
  f.closing_balance_check
FROM tp_finance.gold.fact_contract_monthly_summary s
JOIN (
  SELECT customer_id, contract_id, month,
         CAST(SUM(drawn_this_month) AS DECIMAL(18,2))  AS total_drawn_check,
         CAST(SUM(repaid_this_month) AS DECIMAL(18,2)) AS total_repaid_check,
         CAST(SUM(closing_balance) AS DECIMAL(18,2))   AS closing_balance_check
  FROM tp_finance.gold.fact_facility_monthly
  GROUP BY customer_id, contract_id, month
) f
ON s.customer_id=f.customer_id AND s.contract_id=f.contract_id AND s.month=f.month
WHERE s.total_drawn <> f.total_drawn_check
   OR s.total_repaid <> f.total_repaid_check
   OR s.closing_balance <> f.closing_balance_check;
-- abbove should bring back 0

-- SELECT customer_id, contract_id, facility_id, month, count(*) AS n
-- FROM tp_finance.gold.fact_facility_monthly
-- GROUP BY customer_id, contract_id, facility_id, month
-- HAVING count(*) > 1;
-- -- extra check should bring back 0 - otherwise need to change silve 

-- SELECT customer_id, contract_id, month, count(*) AS n
-- FROM tp_finance.gold.fact_contract_monthly_summary
-- GROUP BY customer_id, contract_id, month
-- HAVING count(*) > 1;

SELECT status, count(*) AS n
FROM tp_finance.audit.generated_documents
GROUP BY status;

SELECT *
FROM tp_finance.audit.generated_documents
ORDER BY generated_ts DESC
LIMIT 20;